In [1]:
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.agents.react.agent import create_react_agent
from langchain import hub
from langchain.agents.agent import AgentExecutor
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv(override=True)

True

In [39]:
from langchain_core.tools import Tool

In [2]:
instruction = """
You are an agent designed to write and execute python code to answer question.
You have access to a python REPL, which you can use to execute code.
If you get an error, debug your code and try again.
Only use the output of your code to answer question.
You might know the answer without running any code, but you should still run the code to get an answer.
If it does not seem like you can write the code to answer question, just return "I don't know" as the answer.
"""

In [3]:
react_prompt = hub.pull("langchain-ai/react-agent-template").partial(instructions = instruction)

In [4]:
tools = [PythonREPLTool()]

In [5]:
llm = ChatOpenAI(model = "gpt-4o")

In [6]:
agent = create_react_agent(llm = llm, 
                           tools = tools, 
                           prompt = react_prompt)

In [7]:
agent_exec = AgentExecutor(agent = agent, max_iteration = 3, tools = tools, verbose= True)

In [9]:
agent_exec.invoke(
    {
        "input": """
                    Generate and save in current working directory 2 QrCodes that points to
                    https://www.linkedin.com/in/ketan-kishore-b89643150/. You have the qrcode package already installed
                    Save the files in a folder called as output_qrs
        """
    }
)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Python_REPL
Action Input: 
```python
import os
import qrcode

# Create the directory if it does not exist
output_dir = "output_qrs"
os.makedirs(output_dir, exist_ok=True)

# URL to encode
url = "https://www.linkedin.com/in/ketan-kishore-b89643150/"

# Generate two QR codes and save them
for i in range(2):
    qr = qrcode.make(url)
    qr.save(os.path.join(output_dir, f"qrcode_{i+1}.png"))
```No tool is necessary now, the QR code files have been successfully generated and saved. 

Final Answer: Two QR codes have been generated and saved in the "output_qrs" directory, each pointing to the URL "https://www.linkedin.com/in/ketan-kishore-b89643150/".

> Finished chain.


{'input': '\n                    Generate and save in current working directory 2 QrCodes that points to\n                    https://www.linkedin.com/in/ketan-kishore-b89643150/. You have the qrcode package already installed\n                    Save the files in a folder called as output_qrs\n        ',
 'output': 'Two QR codes have been generated and saved in the "output_qrs" directory, each pointing to the URL "https://www.linkedin.com/in/ketan-kishore-b89643150/".'}

In [10]:
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent

In [18]:
csv_agent = create_csv_agent(llm = llm, path = "data/episode_info.csv", allow_dangerous_code=True, verbose=True)

In [19]:
csv_agent.invoke({
    "input": "Who wrote the maximum number of episodes"
})



> Entering new AgentExecutor chain...
To find out who wrote the maximum number of episodes, I need to analyze the "Writers" column in the DataFrame. This involves counting the frequency of each writer across all episodes. If an episode is written by multiple writers, they should each be counted separately.

Here’s how I can determine this:

1. Extract the "Writers" column from the DataFrame.
2. Split the writers in each row by a comma to handle episodes that have multiple writers.
3. Count the frequency of each writer in the list obtained.
4. Identify the writer with the highest count.

I'll implement these steps now.

Thought: To find the writer who wrote maximum number of episodes, I need to process the "Writers" column, considering that some episodes might have multiple writers listed together, separated by a comma.
Action: python_repl_ast
Action Input: 
"""
from collections import Counter

# Extract the Writers column
writers_series = df['Writers']

# Split the writers and count 

{'input': 'Who wrote the maximum number of episodes',
 'output': 'Larry David wrote the maximum number of episodes, with a total of 49 episodes.'}

In [20]:
## Router Agent

In [35]:
base_prompt = hub.pull("langchain-ai/react-agent-template").partial(instructions = "")

In [47]:
def python_repl_executor(original_input):
    return agent_exec.invoke({"input": original_input})

In [67]:
tools = [
    Tool(
            name = "Python Code Executor",
            func = python_repl_executor,
            description = """
            useful when you need to transform natural language to python and execute the python code, 
            returning the results of the code executions. 
            DOES NOT ACCEPT CODE AS INPUT
            """
    ),
    Tool(
            name = "CSV Agent",
            func = csv_agent.invoke,
            description = """
            useful when you need to answer question over episode_info.csv file,
            takes an input the entire question and returns the answer after running pandas calculation
            """
    )
]

In [68]:
router_agent = create_react_agent(llm = ChatOpenAI(model = "gpt-4.1"), 
                                  tools = tools, 
                                  prompt = base_prompt)

In [69]:
router_agent_grand = AgentExecutor(agent = router_agent, tools = tools, verbose=True, handle_parsing_errors=True)

In [59]:
router_agent_grand.invoke(
    {
        "input": "What were the maximum number of seasons"
    }
)



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: CSV Agent
Action Input: What is the maximum number of seasons in the data?

> Entering new AgentExecutor chain...

Thought: To find the maximum number of seasons in the data, I need to check the 'Season' column and determine its maximum value. 
Action: python_repl_ast
Action Input: df['Season'].max()9I now know the final answer.

Final Answer: The maximum number of seasons in the data is 9.

> Finished chain.
{'input': 'What is the maximum number of seasons in the data?', 'output': 'The maximum number of seasons in the data is 9.'}Do I need to use a tool? No
Final Answer: The maximum number of seasons in the data is 9.

> Finished chain.


{'input': 'What were the maximum number of seasons',
 'output': 'The maximum number of seasons in the data is 9.'}

In [70]:
router_agent_grand.invoke(
    {
        "input": """
                    Generate and save in current working directory 2 QrCodes that points to
                    https://www.linkedin.com/in/ketan-kishore-b89643150/. You have the qrcode package already installed
                    Save the files in a folder called as output_qrs
                """
    }
)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Python Code Executor
Action Input: Generate and save two QR codes for the LinkedIn URL using the qrcode package, saving the files in a new folder named 'output_qrs' in the current working directory.

> Entering new AgentExecutor chain...
To generate and save two QR codes for a LinkedIn URL using the `qrcode` package, we need to follow these steps:

1. Create a new directory named 'output_qrs'.
2. Use the `qrcode` package to generate two QR codes.
3. Save the QR codes in the 'output_qrs' directory.

I'll write and execute the code to do this. Let's proceed.

```bash
# Make sure you have the qrcode package installed.
# You can install it using: pip install qrcode[pil]
```

```python
Thought: Do I need to use a tool? Yes
Action: Python_REPL
Action Input: 
import os
import qrcode

# Step 1: Create a new directory named 'output_qrs'
output_directory = 'output_qrs'
os.makedirs(output_directory, exist_ok=Tru

{'input': '\n                    Generate and save in current working directory 2 QrCodes that points to\n                    https://www.linkedin.com/in/ketan-kishore-b89643150/. You have the qrcode package already installed\n                    Save the files in a folder called as output_qrs\n                ',
 'output': "The two QR codes pointing to the LinkedIn URL have been successfully generated and saved in a folder named 'output_qrs' in the current working directory. The files are named 'linkedin_qr1.png' and 'linkedin_qr2.png'."}

In [ ]:
router_agent.invoke({
    
})

In [38]:
print(base_prompt.template)

{instructions}

TOOLS:
------

You have access to the following tools:

{tools}

To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```

Begin!

Previous conversation history:
{chat_history}

New input: {input}
{agent_scratchpad}
